# Implementacja modelu uczenia maszynowego na potrzeby predykcji kursu indeksu WIG20

Ideą numer 1 jest próba wyłonienia najbardziej istotnych informacji, które zwiększają prawdopodobieństwo podjęcia właściwej decyzji inwestycyjnej.
Ideą numer 2 jest stworzenie modelu uczenia maszynowego, aby sprawdzić jego przydatność w tradingu. Używana do budowy modeli jest bibliteka scikit-learn.
Idea numer 3 to próba wyłonienia powiązań między wskaźnikami, które z wyprzedzeniem informują o zbliżających się sygnałach inwestycyjnych

### Instalacja niezbędnych bibliotek

Bazuję na danych historycznych pochodzących z serwisu: https://www.dukascopy.com/swiss/pl/marketwatch/historical/
Dukascopy daje możliwość pobierania danych za pomocą specjalnej biblioteki pythonowej, co jest bardzo wygodnym rozwiązaniem. Nazwa biblioteki: dukascopy-python

In [2]:
#pip install dukascopy-python

In [3]:
#pip install --upgrade dukascopy-python

Biblioteka dukascopy-python pobiera dane prosto do struktury danych Pandas DataFrame, więc trzeba zainstalować biblitekę Pandas

In [4]:
#pip install pandas

In [5]:
#pip install --upgrade pandas

Note: you may need to restart the kernel to use updated packages.


Sprawdzenie możliwości biblioteki

Niezbędna będzie malipulacja datami: biblioteka datetime

In [6]:
#pip install datetime

In [7]:
#pip install --upgrade datetime

Skoro ML to biblioteka scikit-learn

In [8]:
#pip install scikit-learn

In [9]:
#pip intall --upgrade scikit-learn

Jakaś biblioteka do analizy technicznej

In [10]:
#pip install pandas-ta

In [11]:
#pip install --upgrade pandas-ta

Note: you may need to restart the kernel to use updated packages.


Pandas TA ma funkcje rozpoznawania formacji świecowych. Aby te funkcje działały trzeba mieć dostęp do biblioteki TA Lib

In [12]:
!python -c "import struct; import sys; print(f'Python {'.'.join(map(str, sys.version_info[:3]))}, {struct.calcsize('P') * 8}-bit')"

Python 3.12.7, 64-bit


Pobierz wersję biblioteki pasującą do Twojej wersji Pythona, przykład: Python **3.12**, **64**-bit-> ta_lib-0.6.4-cp**312**-cp**312**-win_**amd64**.whl

https://github.com/cgohlke/talib-build/releases

In [13]:
#pip install TA-Lib
#!pip install yourPATHtoFILE\ta_lib-0.6.4-cp312-cp312-win_amd64.whl

In [14]:
#pip install --upgrade TA-Lib

### Import bibliotek

In [15]:
from datetime import datetime
#
import dukascopy_python as dcp
#
import pandas as pd
#
import pandas_ta as ta
#
import talib as tl

Wyświetlenie dokumentacji biblioteki dukascopy_python

In [16]:
#help(dcp)

Sprawdzenie listy dostępnych instrumentów

In [17]:
import dukascopy_python.instruments as dcpi

In [18]:
#help(dcpi)

Sprawdzenie możliwości biblioteki Pandas TA

In [19]:
#help(ta)

Sprawdzenie jakie wskaźniki i funkcje są dostępne w bibliotece Pandas-TA

In [20]:
df = pd.DataFrame()
lista_wskaźników = [metoda for metoda in dir(df.ta) if not metoda.startswith('_')]

print(f"Znaleziono {len(lista_wskaźników)} wskaźników i funkcji.")
print(lista_wskaźników)

Znaleziono 157 wskaźników i funkcji.
['aberration', 'above', 'above_value', 'accbands', 'ad', 'adjusted', 'adosc', 'adx', 'alma', 'amat', 'ao', 'aobv', 'apo', 'aroon', 'atr', 'bbands', 'below', 'below_value', 'bias', 'bop', 'brar', 'categories', 'cci', 'cdl_pattern', 'cdl_z', 'cfo', 'cg', 'chop', 'cksp', 'cmf', 'cmo', 'constants', 'coppock', 'cores', 'cross', 'cross_value', 'cti', 'datetime_ordered', 'decay', 'decreasing', 'dema', 'dm', 'donchian', 'dpo', 'ebsw', 'efi', 'ema', 'entropy', 'eom', 'er', 'eri', 'exchange', 'fisher', 'fwma', 'ha', 'hilo', 'hl2', 'hlc3', 'hma', 'hwc', 'hwma', 'ichimoku', 'increasing', 'indicators', 'inertia', 'jma', 'kama', 'kc', 'kdj', 'kst', 'kurtosis', 'kvo', 'last_run', 'linreg', 'log_return', 'long_run', 'macd', 'mad', 'massi', 'mcgd', 'median', 'mfi', 'midpoint', 'midprice', 'mom', 'natr', 'nvi', 'obv', 'ohlc4', 'pdist', 'percent_return', 'pgo', 'ppo', 'psar', 'psl', 'pvi', 'pvo', 'pvol', 'pvr', 'pvt', 'pwma', 'qqe', 'qstick', 'quantile', 'reverse', 'r

### Definicja paramertów dla dla funkcji fetch() z biblioteki dukascopy_python

In [21]:
instrument = 'EUR/USD'
interval = '1DAY'
offer_side = 'B' 
start_date = datetime(2000, 1, 1)
end_date = datetime.today()

### Pobranie danych do Pandas DataFrame

In [22]:
df = dcp.fetch(instrument, interval, offer_side, start_date, end_date)

INFO:DUKASCRIPT:current timestamp :2025-08-04T02:00:00


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7154 entries, 2003-05-04 00:00:00+00:00 to 2025-08-04 00:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    7154 non-null   float64
 1   high    7154 non-null   float64
 2   low     7154 non-null   float64
 3   close   7154 non-null   float64
 4   volume  7154 non-null   float64
dtypes: float64(5)
memory usage: 335.3 KB


Zbiór jest kompletny, zgodnie z oczekiwaniami zawiera dane do dnia poprzedzającego dzień uruchomienia skryptu.
Pora zatem na rozbudowanie zbioru i policzenie interesujących nas statystyk, wskaźników i miar.

Wskaźniki trendu

1. adx (Average Directional Index)
2. amat (Archer Moving Average Trend)
3. aroon (Aroon Indicator)
4. cti (Correlation Trend Indicator)
5. dm (Directional Movement)
6. ebsw (Even Better Sine Wave)
7. er (Efficiency Ratio)
8. ichimoku (Ichimoku Kinko Hyo)
9. linreg (Linear Regression)
10. psar (Parabolic Stop and Reverse)
11. slope (Slope)
12. supertrend (Supertrend)
13. td_seq (TD Sequential)
14. ttm_trend (TTM Trend)
15. vhf (Vertical Horizontal Filter)
16. vortex (Vortex Indicator)

In [24]:
# 1. Average Directional Index, ustawienia standardowe
# Default Inputs: length=14, lensig=lenght, scalar=100, mamode='rma', drift=1, offset=0 <- parametry do modyfikacji
df.ta.adx(append=True)

,ADX_14,DMP_14,DMN_14
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-30 00:00:00+00:00,24.348391,15.824123,37.494199
2025-07-31 00:00:00+00:00,25.512280,15.059176,35.681707
2025-08-01 00:00:00+00:00,24.550656,23.434714,29.856006


In [25]:
# 2. Archer Moving Average Trend, ustawienia standardowe
# Default Inputs: fast=8, slow=21, lookback=2, mamode='ema', offset=0 <- parametry do modyfikacji
df.ta.amat(append=True)

,AMATe_LR_8_21_2,AMATe_SR_8_21_2
timestamp,,
2003-05-04 00:00:00+00:00,0,0
2003-05-05 00:00:00+00:00,0,0
2003-05-06 00:00:00+00:00,0,0
2003-05-07 00:00:00+00:00,0,0
2003-05-08 00:00:00+00:00,0,0
...,...,...
2025-07-30 00:00:00+00:00,0,1
2025-07-31 00:00:00+00:00,0,1
2025-08-01 00:00:00+00:00,0,1


In [26]:
# 3. Aroon Indicator, ustawienia standardowe
# Default Inputs: length=14, scalar=100, talib=True, offset=0 <- parametry do modyfikacji
df.ta.aroon(append=True)

,AROOND_14,AROONU_14,AROONOSC_14
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-30 00:00:00+00:00,100.000000,64.285714,-35.714286
2025-07-31 00:00:00+00:00,92.857143,57.142857,-35.714286
2025-08-01 00:00:00+00:00,100.000000,50.000000,-50.000000


In [27]:
# 4. Correlation Trend Indicator, ustawienia standardowe
# Default Inputs: length=12, offset=0 <- parametry do modyfikacji
df.ta.cti(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00   -0.319069
2025-07-31 00:00:00+00:00   -0.587382
2025-08-01 00:00:00+00:00   -0.675145
2025-08-03 00:00:00+00:00   -0.738721
2025-08-04 00:00:00+00:00   -0.722052
Name: CTI_12, Length: 7154, dtype: float64

In [28]:
# 5. Directional Movement, ustawienia standardowe
# Default Inputs: length=14, mamode='rma', talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.dm(append=True)

,DMP_14,DMN_14
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-30 00:00:00+00:00,0.018385,0.043561
2025-07-31 00:00:00+00:00,0.017071,0.040450
2025-08-01 00:00:00+00:00,0.029482,0.037560


In [29]:
# 6. Even Better Sine Wave, ustawienia standardowe
# Default Inputs: length=40, bars=10, offset=0 <- parametry do modyfikacji
df.ta.ebsw(append=True)

C:\Users\mt242\AppData\Local\Temp\ipykernel_21948\2968336787.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.ta.ebsw(append=True)


timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00   -0.918954
2025-07-31 00:00:00+00:00   -0.984813
2025-08-01 00:00:00+00:00   -0.939068
2025-08-03 00:00:00+00:00   -0.711585
2025-08-04 00:00:00+00:00   -0.221419
Name: EBSW_40_10, Length: 7154, dtype: float64

In [30]:
# 7. Efficiency Ratio, ustawienia standardowe
# Default Inputs: length=10, drift=1, offset=0 <- parametry do modyfikacji
df.ta.er(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    0.364168
2025-07-31 00:00:00+00:00    0.394043
2025-08-01 00:00:00+00:00    0.178887
2025-08-03 00:00:00+00:00    0.256928
2025-08-04 00:00:00+00:00    0.342738
Name: ER_10, Length: 7154, dtype: float64

In [31]:
# 8. Ichimoku Kinko Hyo, ustawienia standardowe
# Default Inputs: tenkan=9, kijun=26, senkou=52, include_chikou=True, offset=0 <- parametry do modyfikacji
df.ta.ichimoku(append=True)

(                              ISA_9    ISB_26     ITS_9    IKS_26   ICS_26
 timestamp                                                                 
 2003-05-04 00:00:00+00:00       NaN       NaN       NaN       NaN  1.17337
 2003-05-05 00:00:00+00:00       NaN       NaN       NaN       NaN  1.16444
 2003-05-06 00:00:00+00:00       NaN       NaN       NaN       NaN  1.18397
 2003-05-07 00:00:00+00:00       NaN       NaN       NaN       NaN  1.16973
 2003-05-08 00:00:00+00:00       NaN       NaN       NaN       NaN  1.16989
 ...                             ...       ...       ...       ...      ...
 2025-07-30 00:00:00+00:00  1.159455  1.142790  1.159475  1.161530      NaN
 2025-07-31 00:00:00+00:00  1.161633  1.144745  1.159475  1.160540      NaN
 2025-08-01 00:00:00+00:00  1.161742  1.144745  1.159010  1.160075      NaN
 2025-08-03 00:00:00+00:00  1.164905  1.144745  1.159010  1.159080      NaN
 2025-08-04 00:00:00+00:00  1.165522  1.144745  1.158075  1.159080      NaN
 
 [7154 row

In [32]:
# 9. Linear Regression, ustawienia standardowe
# Default Inputs: length=10, offset=0 <- parametry do modyfikacji
df.ta.linreg(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.163186
2025-07-31 00:00:00+00:00    1.157406
2025-08-01 00:00:00+00:00    1.156071
2025-08-03 00:00:00+00:00    1.154744
2025-08-04 00:00:00+00:00    1.153460
Name: LR_14, Length: 7154, dtype: float64

In [33]:
# 10. Parabolic Stop and Reverse, ustawienia standardowe
# Default Inputs: af0=0.02, af=0.02, max_af=0.2, offset=0 <- parametry do modyfikacji
df.ta.psar(append=True)

,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2
timestamp,,,,
2003-05-04 00:00:00+00:00,NaN,NaN,0.02,0
2003-05-05 00:00:00+00:00,NaN,1.130090,0.02,1
2003-05-06 00:00:00+00:00,1.120140,NaN,0.02,1
2003-05-07 00:00:00+00:00,1.120140,NaN,0.02,0
2003-05-08 00:00:00+00:00,1.120638,NaN,0.04,0
...,...,...,...,...
2025-07-30 00:00:00+00:00,NaN,1.177399,0.06,0
2025-07-31 00:00:00+00:00,NaN,1.175160,0.06,0
2025-08-01 00:00:00+00:00,NaN,1.173055,0.08,0


In [34]:
# 11. Slope, ustawienia standardowe
# Default Inputs: length=1, as_angle=False, to_degrees=False, vertical=None, offset=0 <- parametry do modyfikacji
df.ta.slope(append=True)

timestamp
2003-05-04 00:00:00+00:00        NaN
2003-05-05 00:00:00+00:00    0.00755
2003-05-06 00:00:00+00:00    0.01310
2003-05-07 00:00:00+00:00   -0.00740
2003-05-08 00:00:00+00:00    0.01326
                              ...   
2025-07-30 00:00:00+00:00   -0.01228
2025-07-31 00:00:00+00:00   -0.00061
2025-08-01 00:00:00+00:00    0.01613
2025-08-03 00:00:00+00:00    0.00046
2025-08-04 00:00:00+00:00   -0.00038
Name: SLOPE_1, Length: 7154, dtype: float64

In [35]:
# 12. Supertrend, ustawienia standardowe
# Default Inputs: length=7, multiplier=3.0, offset=0 <- parametry do modyfikacji
df.ta.supertrend(append=True)

,SUPERT_7_3.0,SUPERTd_7_3.0,SUPERTl_7_3.0,SUPERTs_7_3.0
timestamp,,,,
2003-05-04 00:00:00+00:00,0.000000,1,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,1,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,1,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,1,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,1,NaN,NaN
...,...,...,...,...
2025-07-30 00:00:00+00:00,1.176671,-1,NaN,1.176671
2025-07-31 00:00:00+00:00,1.169674,-1,NaN,1.169674
2025-08-01 00:00:00+00:00,1.169674,-1,NaN,1.169674


In [36]:
# 13. TD Sequential, ustawienia standardowe
# Default Inputs: asint=False, offset=0 <- parametry do modyfikacji
df.ta.td_seq(append=True)

,TD_SEQ_UP,TD_SEQ_DN
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
7149,NaN,NaN
7150,NaN,NaN
7151,NaN,NaN
7152,NaN,NaN


In [37]:
# 14. TTM Trend, ustawienia standardowe
# Default Inputs: length=6, offset=0 <- parametry do modyfikacji
df.ta.ttm_trend(append=True)

,TTM_TRND_6
timestamp,
2003-05-04 00:00:00+00:00,-1
2003-05-05 00:00:00+00:00,-1
2003-05-06 00:00:00+00:00,-1
2003-05-07 00:00:00+00:00,-1
2003-05-08 00:00:00+00:00,-1
...,...
2025-07-30 00:00:00+00:00,-1
2025-07-31 00:00:00+00:00,-1
2025-08-01 00:00:00+00:00,1


In [38]:
# 15. Vertical Horizontal Filter, ustawienia standardowe
# Default Inputs: length=28, drift=1, offset=0 <- parametry do modyfikacji
df.ta.vhf(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    0.409022
2025-07-31 00:00:00+00:00    0.417447
2025-08-01 00:00:00+00:00    0.375969
2025-08-03 00:00:00+00:00    0.374776
2025-08-04 00:00:00+00:00    0.357335
Name: VHF_28, Length: 7154, dtype: float64

In [39]:
# 16. Vortex Indicator, ustawienia standardowe
# Default Inputs: length=14, drift=1, offset=0 <- parametry do modyfikacji
df.ta.vortex(append=True)

,VTXP_14,VTXM_14
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-30 00:00:00+00:00,0.786592,1.090566
2025-07-31 00:00:00+00:00,0.833903,1.182757
2025-08-01 00:00:00+00:00,0.853007,1.089655


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7154 entries, 2003-05-04 00:00:00+00:00 to 2025-08-04 00:00:00+00:00
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   open             7154 non-null   float64
 1   high             7154 non-null   float64
 2   low              7154 non-null   float64
 3   close            7154 non-null   float64
 4   volume           7154 non-null   float64
 5   ADX_14           7127 non-null   float64
 6   DMP_14           7141 non-null   float64
 7   DMN_14           7141 non-null   float64
 8   AMATe_LR_8_21_2  7154 non-null   int32  
 9   AMATe_SR_8_21_2  7154 non-null   int32  
 10  AROOND_14        7140 non-null   float64
 11  AROONU_14        7140 non-null   float64
 12  AROONOSC_14      7140 non-null   float64
 13  CTI_12           7143 non-null   float64
 14  EBSW_40_10       7115 non-null   float64
 15  ER_10            7144 non-null   float64
 16  ISA_9       

Oscylatory i wskaźniki pędu (momentum)

1. ao (Awesome Oscillator)
2. apo (Absolute Price Oscillator)
3. bias (Bias)
4. bop (Balance of Power)
5. brar (AR/BR Ratio)
6. cci (Commodity Channel Index)
7. cfo (Chande Forecast Oscillator)
8. cg (Center of Gravity)
9. cmo (Chande Momentum Oscillator)
10. coppock (Coppock Curve)
11. dpo (Detrended Price Oscillator)
12. eri (Elder Ray Index)
13. fisher (Fisher Transform)
14. inertia (Inertia)
15. kdj (KDJ Indicator)
16. kst (KST Oscillator)
17. macd (Moving Average Convergence Divergence)
18. mom (Momentum)
19. pgo (Pretty Good Oscillator)
20. ppo (Percentage Price Oscillator)
21. psl (Psychological Line)
22. qqe (Quantitative Qualitative Estimation)
23. qstick (QStick)
24. roc (Rate of Change)
25. rsi (Relative Strength Index)
26. rsx (Relative Strength Xtra)
27. rvgi (Relative Vigor Index)
28. smi (Stochastic Momentum Index)
29. stc (Schaff Trend Cycle)
30. stoch (Stochastic Oscillator)
31. stochrsi (Stochastic RSI)
32. thermo (Thermometer)
33. trix (Trix)
34. tsi (True Strength Index)
35. uo (Ultimate Oscillator)
36. willr (Williams' %R)

In [41]:
# 1. Awesome Oscillator, ustawienia standardowe
# Default Inputs: fast=5, slow=34, offset=0 <- parametry do modyfikacji
df.ta.ao(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00   -0.004020
2025-07-31 00:00:00+00:00   -0.009853
2025-08-01 00:00:00+00:00   -0.015054
2025-08-03 00:00:00+00:00   -0.016792
2025-08-04 00:00:00+00:00   -0.016386
Name: AO_5_34, Length: 7154, dtype: float64

In [42]:
#2. Absolute Price Oscillator, ustawienia standardowe
# Default Inputs: fast=12, slow=26, mamode='sma', talib=True, offset=0 <- parametry do modyfikacji
df.ta.apo(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00   -0.003413
2025-07-31 00:00:00+00:00   -0.003550
2025-08-01 00:00:00+00:00   -0.003035
2025-08-03 00:00:00+00:00   -0.002686
2025-08-04 00:00:00+00:00   -0.002885
Name: APO_12_26, Length: 7154, dtype: float64

In [43]:
# 3. Bias, ustawienia standardowe
# Default Inputs: length=26, mamode='sma', drift=1, offset=0 <- parametry do modyfikacji
df.ta.bias(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00   -0.022600
2025-07-31 00:00:00+00:00   -0.021888
2025-08-01 00:00:00+00:00   -0.007371
2025-08-03 00:00:00+00:00   -0.006383
2025-08-04 00:00:00+00:00   -0.006092
Name: BIAS_SMA_26, Length: 7154, dtype: float64

In [44]:
# 4. Balance of Power, ustawienia standardowe
# Default Inputs: scalar=1, talib=True, offset=0 <- parametry do modyfikacji
df.ta.bop(append=True)

timestamp
2003-05-04 00:00:00+00:00   -0.584270
2003-05-05 00:00:00+00:00    0.766834
2003-05-06 00:00:00+00:00    0.736399
2003-05-07 00:00:00+00:00   -0.684310
2003-05-08 00:00:00+00:00    0.633993
                               ...   
2025-07-30 00:00:00+00:00   -0.715201
2025-07-31 00:00:00+00:00   -0.111314
2025-08-01 00:00:00+00:00    0.785297
2025-08-03 00:00:00+00:00   -0.184783
2025-08-04 00:00:00+00:00   -0.097436
Name: BOP, Length: 7154, dtype: float64

In [45]:
# 5. AR/BR Ratio, ustawienia standardowe
# Default Inputs: length=26, scalar=100, drift=1, offset=0 <- parametry do modyfikacji
df.ta.brar(append=True)

,AR_26,BR_26
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-30 00:00:00+00:00,53.048973,53.446514
2025-07-31 00:00:00+00:00,52.324016,52.738161
2025-08-01 00:00:00+00:00,67.287449,67.663175


In [46]:
# 6. Commodity Channel Index, ustawienia standardowe
# Default Inputs: length=14, c=0.015, talib=True, offset=0 <- parametry do modyfikacji
df.ta.cci(append=True)

timestamp
2003-05-04 00:00:00+00:00           NaN
2003-05-05 00:00:00+00:00           NaN
2003-05-06 00:00:00+00:00           NaN
2003-05-07 00:00:00+00:00           NaN
2003-05-08 00:00:00+00:00           NaN
                                ...    
2025-07-30 00:00:00+00:00   -195.555185
2025-07-31 00:00:00+00:00   -182.272667
2025-08-01 00:00:00+00:00    -85.939144
2025-08-03 00:00:00+00:00    -35.846846
2025-08-04 00:00:00+00:00    -40.588397
Name: CCI_14_0.015, Length: 7154, dtype: float64

In [47]:
# 7. Chande Forecast Oscillator, ustawienia standardowe
# Default Inputs: length=9, scalar=100, drift=1, offset=0 <- parametry do modyfikacji
df.ta.cfo(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00   -0.963246
2025-07-31 00:00:00+00:00   -0.243688
2025-08-01 00:00:00+00:00    1.148810
2025-08-03 00:00:00+00:00    1.012462
2025-08-04 00:00:00+00:00    0.745927
Name: CFO_9, Length: 7154, dtype: float64

In [48]:
# 8. Center of Gravity, ustawienia standardowe
# Default Inputs: length=10, offset=0 <- parametry do modyfikacji
df.ta.cg(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00   -5.515420
2025-07-31 00:00:00+00:00   -5.526610
2025-08-01 00:00:00+00:00   -5.526706
2025-08-03 00:00:00+00:00   -5.523624
2025-08-04 00:00:00+00:00   -5.517711
Name: CG_10, Length: 7154, dtype: float64

In [49]:
# 9. Chande Momentum Oscillator, ustawienia standardowe
# Default Inputs: length=14, scalar=100, talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.cmo(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-30 00:00:00+00:00   -39.023509
2025-07-31 00:00:00+00:00   -39.678010
2025-08-01 00:00:00+00:00    -6.978924
2025-08-03 00:00:00+00:00    -6.215260
2025-08-04 00:00:00+00:00    -6.807091
Name: CMO_14, Length: 7154, dtype: float64

In [50]:
# 10. Coppock Curve, ustawienia niestandardowe
# Default Inputs: length=10, fast=11, slow=14 <- oscylator miesięczy
# Dostosowanie oscylatora do interwały dziennego: 
# length=10 -> 10*21=210, fast=11 -> 11*21=231, slow=14 -> 14*21=294
# Standardowe parametry: offset=0
df.ta.coppock(length=210, fast=231, slow=294, append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    6.657408
2025-07-31 00:00:00+00:00    6.712087
2025-08-01 00:00:00+00:00    6.782220
2025-08-03 00:00:00+00:00    6.871527
2025-08-04 00:00:00+00:00    6.955439
Name: COPC_231_294_210, Length: 7154, dtype: float64

In [51]:
# 11. Detrended Price Oscillator, ustawienia standardowe
# Default Inputs: length=1, centered=True, offset=0 <- parametry do modyfikacji
df.ta.dpo(append=True)

timestamp
2003-05-04 00:00:00+00:00   NaN
2003-05-05 00:00:00+00:00   NaN
2003-05-06 00:00:00+00:00   NaN
2003-05-07 00:00:00+00:00   NaN
2003-05-08 00:00:00+00:00   NaN
                             ..
2025-07-30 00:00:00+00:00   NaN
2025-07-31 00:00:00+00:00   NaN
2025-08-01 00:00:00+00:00   NaN
2025-08-03 00:00:00+00:00   NaN
2025-08-04 00:00:00+00:00   NaN
Name: DPO_20, Length: 7154, dtype: float64

In [52]:
# 12. Elder Ray Index, ustawienia standardowe
# Default Inputs: length=14, offset=0 <- parametry do modyfikacji
df.ta.eri(append=True)

,BULLP_13,BEARP_13
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-30 00:00:00+00:00,-0.006669,-0.023839
2025-07-31 00:00:00+00:00,-0.014752,-0.020232
2025-08-01 00:00:00+00:00,-0.000763,-0.021303


In [53]:
# 13. Fisher Transform, ustawienia standardowe
# Default Inputs: length=9, signal=1, offset=0 <- parametry do modyfikacji
df.ta.fisher(append=True)

,FISHERT_9_1,FISHERTs_9_1
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-30 00:00:00+00:00,-0.090478,0.574148
2025-07-31 00:00:00+00:00,-0.695118,-0.090478
2025-08-01 00:00:00+00:00,-1.024670,-0.695118


In [54]:
# 14. Inertia, ustawienia standardowe
# Default Inputs: length=20, rvi_length=14, scalar=100, refined=False, thirds=False, mamode='ema, drift=1, offset=0 <- parametry do modyfikacji
df.ta.inertia(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-30 00:00:00+00:00    50.228606
2025-07-31 00:00:00+00:00    46.410710
2025-08-01 00:00:00+00:00    45.766247
2025-08-03 00:00:00+00:00    46.900996
2025-08-04 00:00:00+00:00    46.146211
Name: INERTIA_20_14, Length: 7154, dtype: float64

In [55]:
# 15. KDJ Indicator, ustawienia standardowe
# Default Inputs: length=9, signal=3, offset=0 <- parametry do modyfikacji
df.ta.kdj(append=True)

,K_9_3,D_9_3,J_9_3
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-30 00:00:00+00:00,29.618388,48.373409,-7.891654
2025-07-31 00:00:00+00:00,21.541587,39.429468,-14.234175
2025-08-01 00:00:00+00:00,30.431887,36.430275,18.435113


In [56]:
# 16. KST Oscillator, ustawienia standardowe
# Default Inputs: roc1=10, roc2=15, roc3=20, roc4=30, sma1=10, sma2=10, sma3=10, sma4=15, signal=9, drift=1, offset=0 <- parametry do modyfikacji
df.ta.kst(append=True)

,KST_10_15_20_30_10_10_10_15,KSTs_9
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-30 00:00:00+00:00,58.998438,482.632269
2025-07-31 00:00:00+00:00,-142.390993,377.939830
2025-08-01 00:00:00+00:00,-243.660464,274.651323


In [57]:
# 17. Moving Average Convergence Divergence, ustawienia standardowe
# Default Inputs: fast=12, slow=26, signal=9, talib=True, offset=0 <- parametry do modyfikacji
df.ta.macd(append=True)

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-30 00:00:00+00:00,-0.001283,-0.002948,0.001666
2025-07-31 00:00:00+00:00,-0.002893,-0.003647,0.000754
2025-08-01 00:00:00+00:00,-0.002836,-0.002872,0.000036


In [58]:
# 18. Momentum, ustawienia standardowe
# Default Inputs: length=1, talib=True, offset=0 <- parametry do modyfikacji
df.ta.mom(append=True)

timestamp
2003-05-04 00:00:00+00:00        NaN
2003-05-05 00:00:00+00:00        NaN
2003-05-06 00:00:00+00:00        NaN
2003-05-07 00:00:00+00:00        NaN
2003-05-08 00:00:00+00:00        NaN
                              ...   
2025-07-30 00:00:00+00:00   -0.01929
2025-07-31 00:00:00+00:00   -0.02077
2025-08-01 00:00:00+00:00   -0.01115
2025-08-03 00:00:00+00:00   -0.01502
2025-08-04 00:00:00+00:00   -0.01895
Name: MOM_10, Length: 7154, dtype: float64

In [59]:
# 19. Pretty Good Oscillator, ustawienia standardowe
# Default Inputs: length=14, offset=0 <- parametry do modyfikacji
df.ta.pgo(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00   -3.037309
2025-07-31 00:00:00+00:00   -2.904610
2025-08-01 00:00:00+00:00   -0.684725
2025-08-03 00:00:00+00:00   -0.591499
2025-08-04 00:00:00+00:00   -0.602819
Name: PGO_14, Length: 7154, dtype: float64

In [60]:
# 20. Percentage Price Oscillator, ustawienia standardowe
# Default Inputs: fast=12, slow=26, signal=9, scalar=100, mamode='sma', talib=True, offset=0 <- parametry do modyfikacji
df.ta.ppo(append=True)

,PPO_12_26_9,PPOh_12_26_9,PPOs_12_26_9
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-30 00:00:00+00:00,-0.291909,-0.094114,-0.197796
2025-07-31 00:00:00+00:00,-0.303976,-0.084944,-0.219032
2025-08-01 00:00:00+00:00,-0.260085,-0.032843,-0.227242


In [61]:
# 21. Psychological Line, ustawienia standardowe
# Default Inputs: length=12, scalar=100, drift=1, offset=0 <- parametry do modyfikacji
df.ta.psl(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-30 00:00:00+00:00    50.000000
2025-07-31 00:00:00+00:00    50.000000
2025-08-01 00:00:00+00:00    50.000000
2025-08-03 00:00:00+00:00    50.000000
2025-08-04 00:00:00+00:00    41.666667
Name: PSL_12, Length: 7154, dtype: float64

In [62]:
# 22. Quantitative Qualitative Estimation, ustawienia standardowe
# Default Inputs: length=14, smooth=5, factor=4.236, mamode='sma', drift=1, offset=0 <- parametry do modyfikacji
df.ta.qqe(append=True)

,QQE_14_5_4.236,QQE_14_5_4.236_RSIMA,QQEl_14_5_4.236,QQEs_14_5_4.236
timestamp,,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN,NaN
...,...,...,...,...
2025-07-30 00:00:00+00:00,49.396291,42.569482,NaN,49.396291
2025-07-31 00:00:00+00:00,45.536914,38.433319,NaN,45.536914
2025-08-01 00:00:00+00:00,45.536914,41.125726,NaN,45.536914


In [63]:
# 23. QStick, ustawienia standardowe
# Default Inputs: length=10, offset=0 <- parametry do modyfikacji
df.ta.qstick(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00   -0.002119
2025-07-31 00:00:00+00:00   -0.002238
2025-08-01 00:00:00+00:00   -0.001274
2025-08-03 00:00:00+00:00   -0.001736
2025-08-04 00:00:00+00:00   -0.002130
Name: QS_10, Length: 7154, dtype: float64

In [64]:
# 24. Rate of Change, ustawienia standardowe
# Default Inputs: length=1, scalar=100, talib=True, offset=0 <- parametry do modyfikacji
df.ta.roc(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00   -1.659969
2025-07-31 00:00:00+00:00   -1.785991
2025-08-01 00:00:00+00:00   -0.953440
2025-08-03 00:00:00+00:00   -1.279627
2025-08-04 00:00:00+00:00   -1.609574
Name: ROC_10, Length: 7154, dtype: float64

In [65]:
# 25. Relative Strength Index, ustawienia standardowe
# Default Inputs: length=14, scalar=100, talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.rsi(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-30 00:00:00+00:00    30.488246
2025-07-31 00:00:00+00:00    30.160995
2025-08-01 00:00:00+00:00    46.510538
2025-08-03 00:00:00+00:00    46.892370
2025-08-04 00:00:00+00:00    46.596455
Name: RSI_14, Length: 7154, dtype: float64

In [66]:
# 26. Relative Strength Xtra, ustawienia standardowe
# Default Inputs: length=14, drift=1, offset=0 <- parametry do modyfikacji
df.ta.rsx(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-30 00:00:00+00:00    40.887112
2025-07-31 00:00:00+00:00    33.789413
2025-08-01 00:00:00+00:00    32.979340
2025-08-03 00:00:00+00:00    34.464178
2025-08-04 00:00:00+00:00    36.417608
Name: RSX_14, Length: 7154, dtype: float64

In [67]:
# 27. Relative Vigor Index, ustawienia standardowe
# Default Inputs: length=14, swma_length=4, offset=0 <- parametry do modyfikacji
df.ta.rvgi(append=True)

,RVGI_14_4,RVGIs_14_4
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-30 00:00:00+00:00,-0.105943,-0.016505
2025-07-31 00:00:00+00:00,-0.160458,-0.070543
2025-08-01 00:00:00+00:00,-0.159437,-0.121055


In [68]:
# 28. Stochastic Momentum Index, ustawienia standardowe
# Default Inputs: fast=5, slow=20, signal=5, scalar=1, offset=0 <- parametry do modyfikacji
df.ta.smi(append=True)

,SMI_5_20_5,SMIs_5_20_5,SMIo_5_20_5
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-30 00:00:00+00:00,-0.274861,-0.070112,-0.204748
2025-07-31 00:00:00+00:00,-0.357900,-0.166042,-0.191858
2025-08-01 00:00:00+00:00,-0.245173,-0.192419,-0.052754


In [69]:
# 29. Schaff Trend Cycle, ustawienia standardowe
# Default Inputs: tclength=10, fast=12, slow=26, factor=0.5, offset=0 <- parametry do modyfikacji
df.ta.stc(append=True)

C:\Users\mt242\AppData\Local\Temp\ipykernel_21948\4005191877.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.ta.stc(append=True)


,STC_10_12_26_0.5,STCmacd_10_12_26_0.5,STCstoch_10_12_26_0.5
timestamp,,,
2003-05-04 00:00:00+00:00,0.000000,NaN,0.000000
2003-05-05 00:00:00+00:00,0.000000,NaN,0.000000
2003-05-06 00:00:00+00:00,0.000000,NaN,0.000000
2003-05-07 00:00:00+00:00,0.000000,NaN,0.000000
2003-05-08 00:00:00+00:00,0.000000,NaN,0.000000
...,...,...,...
2025-07-30 00:00:00+00:00,40.285313,-0.001283,14.302810
2025-07-31 00:00:00+00:00,23.814919,-0.002893,7.151405
2025-08-01 00:00:00+00:00,11.907459,-0.002836,3.575703


In [70]:
# 30. Stochastic Oscillator, ustawienia standardowe
# Default Inputs: k=14, d=3, smooth_k=3, mamode='sma', offset=0 <- parametry do modyfikacji
df.ta.stoch(append=True)

,STOCHk_14_3_3,STOCHd_14_3_3
timestamp,,
2003-05-19 00:00:00+00:00,NaN,NaN
2003-05-20 00:00:00+00:00,NaN,NaN
2003-05-21 00:00:00+00:00,86.540110,NaN
2003-05-22 00:00:00+00:00,88.448066,NaN
2003-05-23 00:00:00+00:00,88.512464,87.833546
...,...,...
2025-07-30 00:00:00+00:00,11.949132,36.887747
2025-07-31 00:00:00+00:00,8.043562,19.390850
2025-08-01 00:00:00+00:00,20.187010,13.393234


In [71]:
# 31. Stochastic RSI, ustawienia standardowe
# Default Inputs: length=14, rsi_length=14, k=3, d=3, mamode='sma', offset=0 <- parametry do modyfikacji
df.ta.stochrsi(append=True)

,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-30 00:00:00+00:00,1.373716e-13,26.314771
2025-07-31 00:00:00+00:00,1.373716e-13,9.086750
2025-08-01 00:00:00+00:00,1.584284e+01,5.280948


In [72]:
# 32. Thermometer, ustawienia standardowe
# Default Inputs: length=20, long=2, short=0.5, mamode='ema', drift=1, offset=0 <- parametry do modyfikacji
df.ta.thermo(append=True)

,THERMO_20_2_0.5,THERMOma_20_2_0.5,THERMOl_20_2_0.5,THERMOs_20_2_0.5
timestamp,,,,
2003-05-04 00:00:00+00:00,NaN,NaN,0,0
2003-05-05 00:00:00+00:00,0.00671,NaN,0,0
2003-05-06 00:00:00+00:00,0.01497,NaN,0,0
2003-05-07 00:00:00+00:00,0.00542,NaN,0,0
2003-05-08 00:00:00+00:00,0.00754,NaN,0,0
...,...,...,...,...
2025-07-30 00:00:00+00:00,0.01180,0.007146,1,1
2025-07-31 00:00:00+00:00,0.01119,0.007531,1,1
2025-08-01 00:00:00+00:00,0.01363,0.008112,1,1


In [73]:
# 33. Trix, ustawienia standardowe
# Default Inputs: length=18, signal=9, scalar=100, drift=1, offset=0 <- parametry do modyfikacji
df.ta.trix(append=True)

,TRIX_30_9,TRIXs_30_9
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-30 00:00:00+00:00,0.062499,0.068726
2025-07-31 00:00:00+00:00,0.059676,0.067149
2025-08-01 00:00:00+00:00,0.056850,0.065414


In [74]:
# 34. True Strength Index, ustawienia standardowe
# Default Inputs: fast=13, slow=25, signal=13, scalar=100, mamode='ema', drift=1, offset=0 <- parametry do modyfikacji
df.ta.tsi(append=True)

,TSI_13_25_13,TSIs_13_25_13
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-30 00:00:00+00:00,-5.168391,10.688194
2025-07-31 00:00:00+00:00,-11.661071,7.495442
2025-08-01 00:00:00+00:00,-10.822467,4.878598


In [75]:
# 35. Ultimate Oscillator, ustawienia standardowe
# Default Inputs: fast=7, medium=14, slow=28, fast_w=4.0, medium_w=2.0, slow_w=1.0, talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.uo(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-30 00:00:00+00:00    37.086152
2025-07-31 00:00:00+00:00    33.705838
2025-08-01 00:00:00+00:00    45.767387
2025-08-03 00:00:00+00:00    44.484806
2025-08-04 00:00:00+00:00    46.395191
Name: UO_7_14_28, Length: 7154, dtype: float64

In [76]:
# 36. Williams' %R, ustawienia standardowe
# Default Inputs: length=14, talib=True, offset=0 <- parametry do modyfikacji
df.ta.willr(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-30 00:00:00+00:00   -93.039443
2025-07-31 00:00:00+00:00   -94.612013
2025-08-01 00:00:00+00:00   -51.787513
2025-08-03 00:00:00+00:00   -50.629406
2025-08-04 00:00:00+00:00   -51.586103
Name: WILLR_14, Length: 7154, dtype: float64

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7154 entries, 2003-05-04 00:00:00+00:00 to 2025-08-04 00:00:00+00:00
Data columns (total 98 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   open                         7154 non-null   float64
 1   high                         7154 non-null   float64
 2   low                          7154 non-null   float64
 3   close                        7154 non-null   float64
 4   volume                       7154 non-null   float64
 5   ADX_14                       7127 non-null   float64
 6   DMP_14                       7141 non-null   float64
 7   DMN_14                       7141 non-null   float64
 8   AMATe_LR_8_21_2              7154 non-null   int32  
 9   AMATe_SR_8_21_2              7154 non-null   int32  
 10  AROOND_14                    7140 non-null   float64
 11  AROONU_14                    7140 non-null   float64
 12  AROONOSC_14                 

Wskaźniki zmienności i kanały

1. aberration (Aberration)
2. accbands (Acceleration Bands)
3. atr (Average True Range)
4. bbands (Bollinger Bands)
5. chop (Choppiness Index)
6. cksp (Chande Kroll Stop)
7. donchian (Donchian Channels)
8. entropy (Information Entropy)
9. hilo (High-Low Channels)
10. kc (Keltner Channels)
11. massi (Mass Index)
12. natr (Normalized Average True Range)
13. rvi (Relative Volatility Index)
14. squeeze (TTM Squeeze)
15. squeeze_pro (TTM Squeeze Pro)
16. tos_stdevall (thinkorswim Standard Deviation All)
17. true_range (True Range)
18. ui (Ulcer Index)

In [78]:
# 1. Aberration, ustawienia standardowe
# Default Inputs: length=5, atr_length=15, offset=0 <- parametry do modyfikacji
df.ta.aberration(append=True)

,ABER_ZG_5_15,ABER_SG_5_15,ABER_XG_5_15,ABER_ATR_5_15
timestamp,,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,1.133375,NaN,NaN,NaN
...,...,...,...,...
2025-07-30 00:00:00+00:00,1.163325,1.171566,1.155083,0.008242
2025-07-31 00:00:00+00:00,1.157223,1.165281,1.149166,0.008058
2025-08-01 00:00:00+00:00,1.152535,1.161424,1.143645,0.008890


In [79]:
# 2. Acceleration Bands, ustawienia standardowe
# Default Inputs: length=10, c=4, mamode='sma', drift=1, offset=0 <- parametry do modyfikacji
df.ta.accbands(append=True)

,ACCBL_20,ACCBM_20,ACCBU_20
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-30 00:00:00+00:00,1.147643,1.166662,1.186866
2025-07-31 00:00:00+00:00,1.146535,1.165155,1.185065
2025-08-01 00:00:00+00:00,1.143510,1.164381,1.185922


In [80]:
# 3. Average True Range, ustawienia standardowe
# Default Inputs: length=14, mamode='rma', talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.atr(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    0.008299
2025-07-31 00:00:00+00:00    0.008097
2025-08-01 00:00:00+00:00    0.008986
2025-08-03 00:00:00+00:00    0.008476
2025-08-04 00:00:00+00:00    0.008149
Name: ATRr_14, Length: 7154, dtype: float64

In [81]:
# 4. Bollinger Bands, ustawienia standardowe
# Default Inputs: length=5, std=2, ddof=0, mamode='sma', talib=True, offset=0 <- parametry do modyfikacji
df.ta.bbands(append=True)

,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
timestamp,,,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,1.116562,1.135282,1.154002,3.297886,0.845029
...,...,...,...,...,...
2025-07-30 00:00:00+00:00,1.136936,1.161394,1.185852,4.211789,0.119466
2025-07-31 00:00:00+00:00,1.130423,1.155036,1.179649,4.261896,0.238636
2025-08-01 00:00:00+00:00,1.136408,1.151588,1.166768,2.636445,0.721073


In [82]:
# 5. Choppiness Index, ustawienia standardowe
# Default Inputs: length=14, atr_length=1, ln=False, scalar=100, drift=1, offset=0 <- parametry do modyfikacji
df.ta.chop(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-30 00:00:00+00:00    44.203085
2025-07-31 00:00:00+00:00    42.808689
2025-08-01 00:00:00+00:00    43.350934
2025-08-03 00:00:00+00:00    41.447274
2025-08-04 00:00:00+00:00    40.780019
Name: CHOP_14_1_100, Length: 7154, dtype: float64

In [83]:
# 6. Chande Kroll Stop, ustawienia standardowe
# Default Inputs: p=10, x=1, q=9, tvmode=True, offset=0 <- parametry do modyfikacji
df.ta.cksp(append=True)

,CKSPl_10_3_20,CKSPs_10_3_20
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-30 00:00:00+00:00,1.162446,1.166168
2025-07-31 00:00:00+00:00,1.162446,1.165203
2025-08-01 00:00:00+00:00,1.162446,1.165203


In [84]:
# 7. Donchian Channels, ustawienia standardowe
# Default Inputs: lower_length=20, upper_length=20, offset=0 <- parametry do modyfikacji
df.ta.donchian(append=True)

,DCL_20_20,DCM_20_20,DCU_20_20
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-30 00:00:00+00:00,1.14008,1.159475,1.17887
2025-07-31 00:00:00+00:00,1.14008,1.159475,1.17887
2025-08-01 00:00:00+00:00,1.13915,1.159010,1.17887


In [85]:
# 8. Information Entropy, ustawienia standardowe
# Default Inputs: length=10, base=2, offset=0 <- parametry do modyfikacji
df.ta.entropy(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    3.320776
2025-07-31 00:00:00+00:00    3.317604
2025-08-01 00:00:00+00:00    3.316120
2025-08-03 00:00:00+00:00    3.314316
2025-08-04 00:00:00+00:00    3.312331
Name: ENTP_10, Length: 7154, dtype: float64

In [86]:
# 9. High-Low Channels, ustawienia standardowe
# Default Inputs: high_length=13, low_length=21, mamode='sma', offset=0 <- parametry do modyfikacji
df.ta.hilo(append=True)

,HILO_13_21,HILOl_13_21,HILOs_13_21
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-30 00:00:00+00:00,1.170605,NaN,1.170605
2025-07-31 00:00:00+00:00,1.168601,NaN,1.168601
2025-08-01 00:00:00+00:00,1.168299,NaN,1.168299


In [87]:
# 10. Keltner Channels, ustawienia standardowe
# Default Inputs: length=20, scalar=2, mamode='ema', offset=0 <- parametry do modyfikacji
df.ta.kc(append=True)

,KCLe_20_2,KCBe_20_2,KCUe_20_2
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-30 00:00:00+00:00,1.147832,1.165084,1.182336
2025-07-31 00:00:00+00:00,1.146249,1.162902,1.179555
2025-08-01 00:00:00+00:00,1.143484,1.162464,1.181443


In [88]:
# 11. Mass Index, ustawienia standardowe
# Default Inputs: fast=9, slow=25, offset=0 <- parametry do modyfikacji
df.ta.massi(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-30 00:00:00+00:00    25.100503
2025-07-31 00:00:00+00:00    25.298507
2025-08-01 00:00:00+00:00    25.605867
2025-08-03 00:00:00+00:00    25.770287
2025-08-04 00:00:00+00:00    25.935272
Name: MASSI_9_25, Length: 7154, dtype: float64

In [89]:
# 12. Normalized Average True Range, ustawienia standardowe
# Default Inputs: length=20, scalar=100, mamode='ema', talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.natr(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    0.726180
2025-07-31 00:00:00+00:00    0.708941
2025-08-01 00:00:00+00:00    0.775798
2025-08-03 00:00:00+00:00    0.731440
2025-08-04 00:00:00+00:00    0.703466
Name: NATR_14, Length: 7154, dtype: float64

In [90]:
# 13. Relative Volatility Index, ustawienia standardowe
# Default Inputs: length=14, scalar=100, refined=False, thirds=False, mamode='ema', drift=1, offset=0 <- parametry do modyfikacji
df.ta.rvi(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-30 00:00:00+00:00    33.707001
2025-07-31 00:00:00+00:00    26.550435
2025-08-01 00:00:00+00:00    41.104084
2025-08-03 00:00:00+00:00    52.110631
2025-08-04 00:00:00+00:00    42.818994
Name: RVI_14, Length: 7154, dtype: float64

In [91]:
# 14. TTM Squeeze, ustawienia standardowe
# Default Inputs: bb_length=20, bb_std=2, kc_length=20, kc_scalar=1.5, mom_length=12, mom_smooth=6, use_tr=True, mamode='sma', offset=0 <- parametry do modyfikacji
df.ta.squeeze(append=True)

,SQZ_20_2.0_20_1.5,SQZ_ON,SQZ_OFF,SQZ_NO
timestamp,,,,
2003-05-04 00:00:00+00:00,NaN,0,0,1
2003-05-05 00:00:00+00:00,NaN,0,0,1
2003-05-06 00:00:00+00:00,NaN,0,0,1
2003-05-07 00:00:00+00:00,NaN,0,0,1
2003-05-08 00:00:00+00:00,NaN,0,0,1
...,...,...,...,...
2025-07-30 00:00:00+00:00,-0.002538,0,1,0
2025-07-31 00:00:00+00:00,-0.006638,0,1,0
2025-08-01 00:00:00+00:00,-0.008173,0,1,0


In [92]:
# 15. TTM Squeeze Pro, ustawienia standardowe
# Default Inputs: bb_length=20, bb_std=2, kc_length=20, kc_scalar_wide=2, kc_scalar_normal=1.5, kc_scalar_narrow=1, mom_length=12, mom_smooth=6, use_tr=True, mamode='sma', offset=0 <- parametry do modyfikacji
df.ta.squeeze_pro(append=True)

,SQZPRO_20_2.0_20_2_1.5_1,SQZPRO_ON_WIDE,SQZPRO_ON_NORMAL,SQZPRO_ON_NARROW,SQZPRO_OFF,SQZPRO_NO
timestamp,,,,,,
2003-05-04 00:00:00+00:00,NaN,0,0,0,0,1
2003-05-05 00:00:00+00:00,NaN,0,0,0,0,1
2003-05-06 00:00:00+00:00,NaN,0,0,0,0,1
2003-05-07 00:00:00+00:00,NaN,0,0,0,0,1
2003-05-08 00:00:00+00:00,NaN,0,0,0,0,1
...,...,...,...,...,...,...
2025-07-30 00:00:00+00:00,-0.002538,0,0,0,1,0
2025-07-31 00:00:00+00:00,-0.006638,0,0,0,1,0
2025-08-01 00:00:00+00:00,-0.008173,0,0,0,1,0


In [93]:
# 16. Think or Swim Standard Deviation All, ustawienia standardowe
# Default Inputs: length=None (means All), stds=[1,2,3], ddof=1, offset=0 <- parametry do modyfikacji
df.ta.tos_stdevall(append=True)

,TOS_STDEVALL_LR,TOS_STDEVALL_L_1,TOS_STDEVALL_U_1,TOS_STDEVALL_L_2,TOS_STDEVALL_U_2,TOS_STDEVALL_L_3,TOS_STDEVALL_U_3
timestamp,,,,,,,
2003-05-04 00:00:00+00:00,1.369000,1.241118,1.496882,1.113237,1.624763,0.985355,1.752645
2003-05-05 00:00:00+00:00,1.368961,1.241079,1.496842,1.113197,1.624724,0.985316,1.752606
2003-05-06 00:00:00+00:00,1.368921,1.241039,1.496803,1.113158,1.624684,0.985276,1.752566
2003-05-07 00:00:00+00:00,1.368882,1.241000,1.496763,1.113118,1.624645,0.985237,1.752527
2003-05-08 00:00:00+00:00,1.368842,1.240960,1.496724,1.113079,1.624605,0.985197,1.752487
...,...,...,...,...,...,...,...
2025-07-30 00:00:00+00:00,1.086749,0.958868,1.214631,0.830986,1.342513,0.703104,1.470394
2025-07-31 00:00:00+00:00,1.086710,0.958828,1.214592,0.830947,1.342473,0.703065,1.470355
2025-08-01 00:00:00+00:00,1.086670,0.958789,1.214552,0.830907,1.342434,0.703025,1.470316


In [94]:
# 17. True Range, ustawienia standardowe
# Default Inputs: talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.true_range(append=True)

timestamp
2003-05-04 00:00:00+00:00        NaN
2003-05-05 00:00:00+00:00    0.00995
2003-05-06 00:00:00+00:00    0.01783
2003-05-07 00:00:00+00:00    0.01058
2003-05-08 00:00:00+00:00    0.02071
                              ...   
2025-07-30 00:00:00+00:00    0.01717
2025-07-31 00:00:00+00:00    0.00548
2025-08-01 00:00:00+00:00    0.02054
2025-08-03 00:00:00+00:00    0.00184
2025-08-04 00:00:00+00:00    0.00390
Name: TRUERANGE_1, Length: 7154, dtype: float64

In [95]:
# 18. Ulcer Index, ustawienia standardowe
# Default Inputs: length=14, scalar=100, offset=0 <- parametry do modyfikacji
df.ta.ui(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.359337
2025-07-31 00:00:00+00:00    1.511260
2025-08-01 00:00:00+00:00    1.524112
2025-08-03 00:00:00+00:00    1.525727
2025-08-04 00:00:00+00:00    1.543462
Name: UI_14, Length: 7154, dtype: float64

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7154 entries, 2003-05-04 00:00:00+00:00 to 2025-08-04 00:00:00+00:00
Columns: 146 entries, open to UI_14
dtypes: float64(131), int32(13), int64(2)
memory usage: 7.9 MB


Wskaźniki wolumenu

1. ad (Accumulation/Distribution Line)
2. adosc (Chaikin Oscillator)
3. aobv (Adaptive On-Balance Volume)
4. cmf (Chaikin Money Flow)
5. efi (Elder's Force Index)
6. eom (Ease of Movement)
7. kvo (Klinger Volume Oscillator)
8. mfi (Money Flow Index)
9. nvi (Negative Volume Index)
10. obv (On-Balance Volume)
11. pvi (Positive Volume Index)
12. pvo (Percentage Volume Oscillator)
13. pvol (Price-Volume)
14. pvr (Price-Volume Rank)
15. pvt (Price and Volume Trend)
16. vp (Volume Profile)
17. vwap (Volume-Weighted Average Price)
18. vwma (Volume-Weighted Moving Average)

In [97]:
# 1. Accumulation/Distribution Line, ustawienia standardowe
# Default Inputs: talib=True, offset=0 <- parametry do modyfikacji
df.ta.ad(append=True)

timestamp
2003-05-04 00:00:00+00:00   -2.310476e+05
2003-05-05 00:00:00+00:00    8.269470e+05
2003-05-06 00:00:00+00:00    1.717434e+06
2003-05-07 00:00:00+00:00    9.957342e+05
2003-05-08 00:00:00+00:00    1.947964e+06
                                 ...     
2025-07-30 00:00:00+00:00    3.312310e+07
2025-07-31 00:00:00+00:00    3.308263e+07
2025-08-01 00:00:00+00:00    3.321027e+07
2025-08-03 00:00:00+00:00    3.321062e+07
2025-08-04 00:00:00+00:00    3.328638e+07
Name: AD, Length: 7154, dtype: float64

In [98]:
# 2. Chaikin Oscillator, ustawienia standardowe
# Default Inputs: fast=12, slow=26, talib=True, offset=0 <- parametry do modyfikacji
df.ta.adosc(append=True)

timestamp
2003-05-04 00:00:00+00:00             NaN
2003-05-05 00:00:00+00:00             NaN
2003-05-06 00:00:00+00:00             NaN
2003-05-07 00:00:00+00:00             NaN
2003-05-08 00:00:00+00:00             NaN
                                 ...     
2025-07-30 00:00:00+00:00   -48526.485354
2025-07-31 00:00:00+00:00   -74056.823320
2025-08-01 00:00:00+00:00   -37156.094359
2025-08-03 00:00:00+00:00   -18571.114373
2025-08-04 00:00:00+00:00    14825.605064
Name: ADOSC_3_10, Length: 7154, dtype: float64

In [99]:
# 3. Adaptive On-Balance Volume, ustawienia standardowe
# Default Inputs: fast=4, slow=12, max_lookback=2, min_lookback=2, mamode='ema', offset=0 <- parametry do modyfikacji
df.ta.aobv(append=True)

,OBV,OBV_min_2,OBV_max_2,OBVe_4,OBVe_12,AOBV_LR_2,AOBV_SR_2
timestamp,,,,,,,
2003-05-04 00:00:00+00:00,2.570405e+05,NaN,NaN,NaN,NaN,0,0
2003-05-05 00:00:00+00:00,1.533046e+06,2.570405e+05,1.533046e+06,NaN,NaN,0,0
2003-05-06 00:00:00+00:00,2.814513e+06,1.533046e+06,2.814513e+06,NaN,NaN,0,0
2003-05-07 00:00:00+00:00,1.541916e+06,1.541916e+06,2.814513e+06,1.536629e+06,NaN,0,0
2003-05-08 00:00:00+00:00,2.808498e+06,1.541916e+06,2.808498e+06,2.045377e+06,NaN,0,0
...,...,...,...,...,...,...,...
2025-07-30 00:00:00+00:00,1.497209e+07,1.497209e+07,1.512620e+07,1.512848e+07,1.524661e+07,0,1
2025-07-31 00:00:00+00:00,1.487565e+07,1.487565e+07,1.497209e+07,1.502735e+07,1.518954e+07,0,1
2025-08-01 00:00:00+00:00,1.502327e+07,1.487565e+07,1.502327e+07,1.502572e+07,1.516396e+07,0,1


In [100]:
# 4. Chaikin Money Flow, ustawienia standardowe
# Default Inputs: length=20, offset=0 <- parametry do modyfikacji
df.ta.cmf(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00   -0.071482
2025-07-31 00:00:00+00:00   -0.094926
2025-08-01 00:00:00+00:00   -0.065246
2025-08-03 00:00:00+00:00   -0.062999
2025-08-04 00:00:00+00:00   -0.009436
Name: CMF_20, Length: 7154, dtype: float64

In [101]:
# 5. Elder's Force Index, ustawienia standardowe
# Default Inputs: length=13, mamode='ema', drift=1, offset=0 <- parametry do modyfikacji
df.ta.efi(append=True)

timestamp
2003-05-04 00:00:00+00:00           NaN
2003-05-05 00:00:00+00:00           NaN
2003-05-06 00:00:00+00:00           NaN
2003-05-07 00:00:00+00:00           NaN
2003-05-08 00:00:00+00:00           NaN
                                ...    
2025-07-30 00:00:00+00:00   -464.054832
2025-07-31 00:00:00+00:00   -406.165051
2025-08-01 00:00:00+00:00     -7.985281
2025-08-03 00:00:00+00:00     -6.421186
2025-08-04 00:00:00+00:00    -10.996853
Name: EFI_13, Length: 7154, dtype: float64

In [102]:
# 6. Ease of Movement, ustawienia standardowe
# Default Inputs: length=14, divisor=100000000, drift=1, offset=0 <- parametry do modyfikacji
df.ta.eom(append=True)

timestamp
2003-05-04 00:00:00+00:00   NaN
2003-05-05 00:00:00+00:00   NaN
2003-05-06 00:00:00+00:00   NaN
2003-05-07 00:00:00+00:00   NaN
2003-05-08 00:00:00+00:00   NaN
                             ..
2025-07-30 00:00:00+00:00   NaN
2025-07-31 00:00:00+00:00   NaN
2025-08-01 00:00:00+00:00   NaN
2025-08-03 00:00:00+00:00   NaN
2025-08-04 00:00:00+00:00   NaN
Name: EOM_14_100000000, Length: 7154, dtype: float64

In [103]:
# 7. Klinger Volume Oscillator, ustawienia standardowe
# Default Inputs: fast=34, slow=55, signal=13, mamode='ema', drift=1, offset=0 <- parametry do modyfikacji
df.ta.kvo(append=True)

,KVO_34_55_13,KVOs_34_55_13
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-30 00:00:00+00:00,-10618.301433,-6338.355442
2025-07-31 00:00:00+00:00,-12065.313809,-7156.492351
2025-08-01 00:00:00+00:00,-8126.527890,-7295.068857


In [104]:
# 8. Money Flow Index, ustawienia standardowe
# Default Inputs: length=14, talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.mfi(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-30 00:00:00+00:00    46.466086
2025-07-31 00:00:00+00:00    47.391417
2025-08-01 00:00:00+00:00    47.259806
2025-08-03 00:00:00+00:00    52.072907
2025-08-04 00:00:00+00:00    44.328414
Name: MFI_14, Length: 7154, dtype: float64

In [105]:
# 9. Negative Volume Index, ustawienia standardowe
# Default Inputs: length=13, initial=1000, offset=0 <- parametry do modyfikacji
df.ta.nvi(append=True)

timestamp
2003-05-04 00:00:00+00:00    1000.000000
2003-05-05 00:00:00+00:00    1000.000000
2003-05-06 00:00:00+00:00    1000.000000
2003-05-07 00:00:00+00:00     999.352207
2003-05-08 00:00:00+00:00    1000.520551
                                ...     
2025-07-30 00:00:00+00:00     990.743448
2025-07-31 00:00:00+00:00     990.690069
2025-08-01 00:00:00+00:00     990.690069
2025-08-03 00:00:00+00:00     990.729783
2025-08-04 00:00:00+00:00     990.729783
Name: NVI_1, Length: 7154, dtype: float64

In [106]:
# 10. On-Balance Volume, ustawienia standardowe
# Default Inputs: talib=True, offset=0 <- parametry do modyfikacji
df.ta.obv(append=True)

timestamp
2003-05-04 00:00:00+00:00    2.570405e+05
2003-05-05 00:00:00+00:00    1.533046e+06
2003-05-06 00:00:00+00:00    2.814513e+06
2003-05-07 00:00:00+00:00    1.541916e+06
2003-05-08 00:00:00+00:00    2.808498e+06
                                 ...     
2025-07-30 00:00:00+00:00    1.497209e+07
2025-07-31 00:00:00+00:00    1.487565e+07
2025-08-01 00:00:00+00:00    1.502327e+07
2025-08-03 00:00:00+00:00    1.502971e+07
2025-08-04 00:00:00+00:00    1.492853e+07
Name: OBV, Length: 7154, dtype: float64

In [107]:
# 11. Positive Volume Index, ustawienia standardowe
# Default Inputs: length=13, initial=1000, offset=0 <- parametry do modyfikacji
df.ta.pvi(append=True)

timestamp
2003-05-04 00:00:00+00:00    1000.000000
2003-05-05 00:00:00+00:00    1000.673091
2003-05-06 00:00:00+00:00    1001.833164
2003-05-07 00:00:00+00:00    1001.833164
2003-05-08 00:00:00+00:00    1001.833164
                                ...     
2025-07-30 00:00:00+00:00    1020.231657
2025-07-31 00:00:00+00:00    1020.231657
2025-08-01 00:00:00+00:00    1021.643881
2025-08-03 00:00:00+00:00    1021.643881
2025-08-04 00:00:00+00:00    1021.611087
Name: PVI_1, Length: 7154, dtype: float64

In [108]:
# 12. Percentage Volume Oscillator, ustawienia standardowe
# Default Inputs: fast=12, slow=26, signal=9, scalar=100, offset=0s <- parametry do modyfikacji
df.ta.pvo(append=True)

,PVO_12_26_9,PVOh_12_26_9,PVOs_12_26_9
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-30 00:00:00+00:00,1.447892,7.441213,-5.993321
2025-07-31 00:00:00+00:00,1.780832,6.219323,-4.438490
2025-08-01 00:00:00+00:00,6.245571,8.547249,-2.301678


In [109]:
# 13. Price-Volume, ustawienia standardowe
# Default Inputs: offset=0 <- parametry do modyfikacji
df.ta.pvol(append=True)

timestamp
2003-05-04 00:00:00+00:00    2.883198e+05
2003-05-05 00:00:00+00:00    1.440917e+06
2003-05-06 00:00:00+00:00    1.463871e+06
2003-05-07 00:00:00+00:00    1.444321e+06
2003-05-08 00:00:00+00:00    1.454289e+06
                                 ...     
2025-07-30 00:00:00+00:00    1.761137e+05
2025-07-31 00:00:00+00:00    1.101471e+05
2025-08-01 00:00:00+00:00    1.709870e+05
2025-08-03 00:00:00+00:00    7.464883e+03
2025-08-04 00:00:00+00:00    1.172124e+05
Name: PVOL, Length: 7154, dtype: float64

In [110]:
# 14. Price-Volume Rank, ustawienia standardowe
# Default Inputs: None
df.ta.pvr(append=True)

timestamp
2003-05-04 00:00:00+00:00    1.0
2003-05-05 00:00:00+00:00    1.0
2003-05-06 00:00:00+00:00    1.0
2003-05-07 00:00:00+00:00    4.0
2003-05-08 00:00:00+00:00    2.0
                            ... 
2025-07-30 00:00:00+00:00    3.0
2025-07-31 00:00:00+00:00    4.0
2025-08-01 00:00:00+00:00    1.0
2025-08-03 00:00:00+00:00    2.0
2025-08-04 00:00:00+00:00    3.0
Name: PVR, Length: 7154, dtype: float64

In [111]:
# 15. Price and Volume Trend, ustawienia standardowe
# Default Inputs: drift=1, offset=0 <- parametry do modyfikacji
df.ta.pvt(append=True)

timestamp
2003-05-04 00:00:00+00:00             NaN
2003-05-05 00:00:00+00:00    8.588685e+05
2003-05-06 00:00:00+00:00    2.345463e+06
2003-05-07 00:00:00+00:00    1.521084e+06
2003-05-08 00:00:00+00:00    3.000886e+06
                                 ...     
2025-07-30 00:00:00+00:00    3.158598e+07
2025-07-31 00:00:00+00:00    3.158084e+07
2025-08-01 00:00:00+00:00    3.178931e+07
2025-08-03 00:00:00+00:00    3.178956e+07
2025-08-04 00:00:00+00:00    3.178624e+07
Name: PVT, Length: 7154, dtype: float64

In [112]:
# 16. Volume Profile, ustawienia standardowe
# Default Inputs: width=10 <- parametry do modyfikacji
df.ta.vp(append=True)

C:\Users\mt242\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,low_close,mean_close,high_close,pos_volume,neg_volume,total_volume
0,1.08022,1.226915,1.36212,4.040275e+08,3.943613e+08,7.983887e+08
1,1.16703,1.289190,1.48710,3.786116e+08,3.599534e+08,7.385650e+08
2,1.24380,1.430158,1.59721,1.577344e+08,1.784314e+08,3.361658e+08
3,1.19208,1.353361,1.48402,5.282053e+07,4.717448e+07,9.999501e+07
4,1.20612,1.312379,1.38771,4.991009e+07,4.805219e+07,9.796228e+07
5,1.04837,1.188945,1.39292,5.515814e+07,6.060427e+07,1.157624e+08
6,1.03906,1.143408,1.25105,9.453988e+07,9.325227e+07,1.877922e+08
7,1.06544,1.132716,1.23382,9.893458e+07,9.740424e+07,1.963388e+08
8,0.95953,1.111788,1.22468,7.754650e+07,6.871864e+07,1.462651e+08
9,1.02400,1.088555,1.18050,6.721492e+07,7.361745e+07,1.408324e+08


In [113]:
# 17. Volume-Weighted Average Price, ustawienia standardowe
# Default Inputs: anchor="D", offset=0 <- parametry do modyfikacji
df.ta.vwap(append=True)

C:\Users\mt242\AppData\Local\Temp\ipykernel_21948\3518576301.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df.ta.vwap(append=True)


timestamp
2003-05-04 00:00:00+00:00    1.122223
2003-05-05 00:00:00+00:00    1.126490
2003-05-06 00:00:00+00:00    1.138210
2003-05-07 00:00:00+00:00    1.136940
2003-05-08 00:00:00+00:00    1.143010
                               ...   
2025-07-30 00:00:00+00:00    1.146703
2025-07-31 00:00:00+00:00    1.142937
2025-08-01 00:00:00+00:00    1.152380
2025-08-03 00:00:00+00:00    1.158727
2025-08-04 00:00:00+00:00    1.157407
Name: VWAP_D, Length: 7154, dtype: float64

In [114]:
# 18. Volume-Weighted Moving Average, ustawienia standardowe
# Default Inputs: length=10, offset=0 <- parametry do modyfikacji
df.ta.vwma(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.163721
2025-07-31 00:00:00+00:00    1.161384
2025-08-01 00:00:00+00:00    1.160188
2025-08-03 00:00:00+00:00    1.158985
2025-08-04 00:00:00+00:00    1.156924
Name: VWMA_10, Length: 7154, dtype: float64

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7154 entries, 2003-05-04 00:00:00+00:00 to 2025-08-04 00:00:00+00:00
Columns: 177 entries, open to VWMA_10
dtypes: float64(160), int32(15), int64(2)
memory usage: 9.6 MB


Funkcje rozpoznające formacje świecowe

1. cdl_pattern (Candlestick Pattern)
2. cdl_z (Candlestick Z-Score)

In [116]:
# 1. Candlestick Pattern, ustawienia standardowe
# Default Inputs: name='all', scalar=100, offset=0 <- parametry do modyfikacji
df.ta.cdl_pattern(name="all", append=True)

,CDL_2CROWS,CDL_3BLACKCROWS,CDL_3INSIDE,CDL_3LINESTRIKE,CDL_3OUTSIDE,CDL_3STARSINSOUTH,CDL_3WHITESOLDIERS,CDL_ABANDONEDBABY,CDL_ADVANCEBLOCK,CDL_BELTHOLD,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
timestamp,,,,,,,,,,,,,,,,,,,,,
2003-05-04 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2003-05-05 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2003-05-06 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2003-05-07 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2003-05-08 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-30 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-07-31 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-08-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
# 2. Candlestick Z-Score, ustawienia standardowe
# Default Inputs: length=10, full=False, ddof=None, offset=0 <- parametry do modyfikacji
df.ta.cdl_z(name='all', append=True)

,open_Z_30_1,high_Z_30_1,low_Z_30_1,close_Z_30_1
timestamp,,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN,NaN
...,...,...,...,...
2025-07-30 00:00:00+00:00,-2.346871,-2.605592,-3.262479,-3.272635
2025-07-31 00:00:00+00:00,-3.250805,-3.334606,-2.681196,-2.774373
2025-08-01 00:00:00+00:00,-2.760798,-1.496976,-2.438864,-1.030557


In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7154 entries, 2003-05-04 00:00:00+00:00 to 2025-08-04 00:00:00+00:00
Columns: 243 entries, open to close_Z_30_1
dtypes: float64(225), int32(16), int64(2)
memory usage: 13.1 MB


Średnie Kroczące, filtry i funkcje wygładzające średnie

1. alma (Arnaud Legoux Moving Average)
2. dema (Double Exponential Moving Average)
3. ema (Exponential Moving Average)
4. fwma (Fibonacci's Weighted Moving Average)
5. hma (Hull Moving Average)
6. hwma (Holt-Winters Moving Average)
7. jma (Jurik Moving Average)
8. kama (Kaufman's Adaptive Moving Average)
9. mcgd (McGinley Dynamic)
10. pwma (Pascal's Weighted Moving Average)
11. rma (Running Moving Average)
12. sinwma (Sine Weighted Moving Average)
13. sma (Simple Moving Average)
14. ssf (Ehlers Super Smoother Filter)
15. swma (Symmetric Weighted Moving Average)
16. t3 (T3 Moving Average)
17. tema (Triple Exponential Moving Average)
18. trima (Triangular Moving Average)
19. vidya (Variable Index Dynamic Average)
20. wma (Weighted Moving Average)
21. zlma (Zero Lag Moving Average)

In [119]:
# 1. Arnaud Legoux Moving Average, ustawienia standardowe
# Default Inputs: length=10, sigma=6.0, distribution_offset=0, offset=None <- parametry do modyfikacji
df.ta.alma(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.171195
2025-07-31 00:00:00+00:00    1.174067
2025-08-01 00:00:00+00:00    1.174839
2025-08-03 00:00:00+00:00    1.173504
2025-08-04 00:00:00+00:00    1.169801
Name: ALMA_10_6.0_0.85, Length: 7154, dtype: float64

In [120]:
# 2. Double Exponential Moving Average, ustawienia standardowe
# Default Inputs: length=10, talib=True, offset=0 <- parametry do modyfikacji
df.ta.dema(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.157329
2025-07-31 00:00:00+00:00    1.151537
2025-08-01 00:00:00+00:00    1.152682
2025-08-03 00:00:00+00:00    1.153787
2025-08-04 00:00:00+00:00    1.154565
Name: DEMA_10, Length: 7154, dtype: float64

In [121]:
# 3. Exponential Moving Average, ustawienia standardowe
# Default Inputs: length=10, talib=True, offset=0 <- parametry do modyfikacji
df.ta.ema(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.162578
2025-07-31 00:00:00+00:00    1.158867
2025-08-01 00:00:00+00:00    1.158764
2025-08-03 00:00:00+00:00    1.158763
2025-08-04 00:00:00+00:00    1.158694
Name: EMA_10, Length: 7154, dtype: float64

In [122]:
# 4. Fibonacci's Weighted Moving Average, ustawienia standardowe
# Default Inputs: length=10, asc=True, offset=0 <- parametry do modyfikacji
df.ta.fwma(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.155525
2025-07-31 00:00:00+00:00    1.150375
2025-08-01 00:00:00+00:00    1.153378
2025-08-03 00:00:00+00:00    1.155396
2025-08-04 00:00:00+00:00    1.156491
Name: FWMA_10, Length: 7154, dtype: float64

In [123]:
# 5. Hull Moving Average, ustawienia standardowe
# Default Inputs: length=10, offset=0 <- parametry do modyfikacji
df.ta.hma(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.155961
2025-07-31 00:00:00+00:00    1.146248
2025-08-01 00:00:00+00:00    1.143095
2025-08-03 00:00:00+00:00    1.145613
2025-08-04 00:00:00+00:00    1.150681
Name: HMA_10, Length: 7154, dtype: float64

In [124]:
# 6. Holt-Winters Moving Average, ustawienia standardowe
# Default Inputs: na=0.2, nb=0.1, nc=0.1, offset=0 <- parametry do modyfikacji
df.ta.hwma(append=True)

timestamp
2003-05-04 00:00:00+00:00    1.121690
2003-05-05 00:00:00+00:00    1.123359
2003-05-06 00:00:00+00:00    1.127726
2003-05-07 00:00:00+00:00    1.129942
2003-05-08 00:00:00+00:00    1.134814
                               ...   
2025-07-30 00:00:00+00:00    1.152683
2025-07-31 00:00:00+00:00    1.147704
2025-08-01 00:00:00+00:00    1.147004
2025-08-03 00:00:00+00:00    1.146637
2025-08-04 00:00:00+00:00    1.146392
Name: HWMA_0.2_0.1_0.1, Length: 7154, dtype: float64

In [125]:
# 7. Jurik Moving Average, ustawienia standardowe
# Default Inputs: length=7, phase=0, offset=0 <- parametry do modyfikacji
df.ta.jma(append=True)

C:\Users\mt242\AppData\Local\Temp\ipykernel_21948\1544405560.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.ta.jma(append=True)


timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.151955
2025-07-31 00:00:00+00:00    1.145316
2025-08-01 00:00:00+00:00    1.146956
2025-08-03 00:00:00+00:00    1.150443
2025-08-04 00:00:00+00:00    1.153319
Name: JMA_7_0, Length: 7154, dtype: float64

In [126]:
# 8. Kaufman's Adaptive Moving Average, ustawienia standardowe
# Default Inputs: length=10, fast=2, slow=30, drift=1, offset=0 <- parametry do modyfikacji
df.ta.kama(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.166050
2025-07-31 00:00:00+00:00    1.163875
2025-08-01 00:00:00+00:00    1.163710
2025-08-03 00:00:00+00:00    1.163472
2025-08-04 00:00:00+00:00    1.163098
Name: KAMA_10_2_30, Length: 7154, dtype: float64

In [128]:
# 9. McGinley Dynamic, ustawienia standardowe
# Default Inputs: length=10, offset=0, c=1 <- parametry do modyfikacji
# df.ta.mcgd(append=True)
# parametr append=True wyrzuca błędy, więc dodajemy do df kolumnę "na piechotę", ale to też wyrzuca błędy. Na potrzeby
# df[MCGD] = df.ta.mcgd()
# print(df)
# pomijam dodanie tego wskaźnika, nie powinien on wiele zmienić w budowaniu całego modelu.

In [129]:
# 10. Pascal's Weighted Moving Average, ustawienia standardowe
# Default Inputs: length=10, asc=True, offset=0 <- parametry do modyfikacji
df.ta.pwma(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.173625
2025-07-31 00:00:00+00:00    1.170528
2025-08-01 00:00:00+00:00    1.165103
2025-08-03 00:00:00+00:00    1.158419
2025-08-04 00:00:00+00:00    1.152784
Name: PWMA_10, Length: 7154, dtype: float64

In [130]:
# 11. Running Moving Average, ustawienia standardowe
# Default Inputs: length=10, offset=0 <- parametry do modyfikacji
df.ta.rma(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.165034
2025-07-31 00:00:00+00:00    1.162748
2025-08-01 00:00:00+00:00    1.162303
2025-08-03 00:00:00+00:00    1.161949
2025-08-04 00:00:00+00:00    1.161592
Name: RMA_10, Length: 7154, dtype: float64

In [131]:
# 12. Sine Weighted Moving Average, ustawienia standardowe
# Default Inputs: length=10, offset=0 <- parametry do modyfikacji
df.ta.sinwma(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.168303
2025-07-31 00:00:00+00:00    1.167513
2025-08-01 00:00:00+00:00    1.166372
2025-08-03 00:00:00+00:00    1.165004
2025-08-04 00:00:00+00:00    1.163413
Name: SINWMA_14, Length: 7154, dtype: float64

In [132]:
# 13. Simple Moving Average, ustawienia standardowe
# Default Inputs: length=10, talib=True, offset=0 <- parametry do modyfikacji
df.ta.sma(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.166592
2025-07-31 00:00:00+00:00    1.164515
2025-08-01 00:00:00+00:00    1.163400
2025-08-03 00:00:00+00:00    1.161898
2025-08-04 00:00:00+00:00    1.160003
Name: SMA_10, Length: 7154, dtype: float64

In [133]:
# 14. Ehlers Super Smoother Filter, ustawienia standardowe
# Default Inputs: length=10, poles=2, offset=0 <- parametry do modyfikacji
df.ta.ssf(append=True)

timestamp
2003-05-04 00:00:00+00:00    1.148934
2003-05-05 00:00:00+00:00    1.140063
2003-05-06 00:00:00+00:00    1.136991
2003-05-07 00:00:00+00:00    1.135209
2003-05-08 00:00:00+00:00    1.137765
                               ...   
2025-07-30 00:00:00+00:00    1.157744
2025-07-31 00:00:00+00:00    1.150403
2025-08-01 00:00:00+00:00    1.149383
2025-08-03 00:00:00+00:00    1.151338
2025-08-04 00:00:00+00:00    1.153925
Name: SSF_10_2, Length: 7154, dtype: float64

In [134]:
# 15. Symmetric Weighted Moving Average, ustawienia standardowe
# Default Inputs: length=10, asc=True, offset=0 <- parametry do modyfikacji
df.ta.swma(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.170450
2025-07-31 00:00:00+00:00    1.167658
2025-08-01 00:00:00+00:00    1.163923
2025-08-03 00:00:00+00:00    1.159957
2025-08-04 00:00:00+00:00    1.156573
Name: SWMA_10, Length: 7154, dtype: float64

In [135]:
# 16. T3 Moving Average, ustawienia standardowe
# Default Inputs: length=10, a=0.7, talib=True, offset=0 <- parametry do modyfikacji
df.ta.t3(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.168275
2025-07-31 00:00:00+00:00    1.166578
2025-08-01 00:00:00+00:00    1.164583
2025-08-03 00:00:00+00:00    1.162622
2025-08-04 00:00:00+00:00    1.160865
Name: T3_10_0.7, Length: 7154, dtype: float64

In [136]:
# 17. Triple Exponential Moving Average, ustawienia standardowe
# Default Inputs: length=10, talib=True, offset=0 <- parametry do modyfikacji
df.ta.tema(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.152818
2025-07-31 00:00:00+00:00    1.146143
2025-08-01 00:00:00+00:00    1.149291
2025-08-03 00:00:00+00:00    1.151916
2025-08-04 00:00:00+00:00    1.153728
Name: TEMA_10, Length: 7154, dtype: float64

In [137]:
# 18. Triangular Moving Average, ustawienia standardowe
# Default Inputs: length=10, talib=True, offset=0 <- parametry do modyfikacji
df.ta.trima(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.170450
2025-07-31 00:00:00+00:00    1.167658
2025-08-01 00:00:00+00:00    1.163923
2025-08-03 00:00:00+00:00    1.159957
2025-08-04 00:00:00+00:00    1.156573
Name: TRIMA_10, Length: 7154, dtype: float64

In [138]:
# 19. Variable Index Dynamic Average, ustawienia standardowe
# Default Inputs: length=14, drift=1, offset=0 <- parametry do modyfikacji
df.ta.vidya(append=True)

C:\Users\mt242\AppData\Local\Temp\ipykernel_21948\4149044402.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.07676862792912949' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.ta.vidya(append=True)


timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.160825
2025-07-31 00:00:00+00:00    1.160044
2025-08-01 00:00:00+00:00    1.160028
2025-08-03 00:00:00+00:00    1.160021
2025-08-04 00:00:00+00:00    1.160010
Name: VIDYA_14, Length: 7154, dtype: float64

In [139]:
# 20. Weighted Moving Average, ustawienia standardowe
# Default Inputs: length=10, asc=True, talib=True, offset=0 <- parametry do modyfikacji
df.ta.wma(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.163321
2025-07-31 00:00:00+00:00    1.158881
2025-08-01 00:00:00+00:00    1.157751
2025-08-03 00:00:00+00:00    1.156907
2025-08-04 00:00:00+00:00    1.156268
Name: WMA_10, Length: 7154, dtype: float64

In [140]:
# 21. Zero Lag Moving Average, ustawienia standardowe
# Default Inputs: length=10, mamode='ema', offset=0 <- parametry do modyfikacji
df.ta.zlma(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.154130
2025-07-31 00:00:00+00:00    1.145888
2025-08-01 00:00:00+00:00    1.147903
2025-08-03 00:00:00+00:00    1.150550
2025-08-04 00:00:00+00:00    1.154810
Name: ZL_EMA_10, Length: 7154, dtype: float64

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7154 entries, 2003-05-04 00:00:00+00:00 to 2025-08-04 00:00:00+00:00
Columns: 263 entries, open to ZL_EMA_10
dtypes: float64(245), int32(16), int64(2)
memory usage: 14.2 MB


Miary statystyczne i pomocnicze

1. decay (Decay)
2. hwc / wcp (Heavily Weighted Close / Weighted Close Price)
3. hl2 (High-Low 2)
4. hlc3 (High-Low-Close 3)
5. kurtosis (Kurtoza)
6. log_return (Logarithmic Return)
7. mad (Mean Absolute Deviation)
8. median (Mediana)
9. midpoint (Midpoint)
10. midprice (Midprice)
11. ohlc4 (Open-High-Low-Close 4)
12. pdist (Price Distance)
13. percent_return (Percent Return)
14. quantile (Kwantyl)
15. skew (Skośność)
16. stdev (Standard Deviation)
17. variance (Wariancja)
18. zscore (Z-Score)

In [144]:
# 1. Decay, ustawienia standardowe
# Default Inputs: kind='linear', length=5, mode='linear', offset=0 <- parametry do modyfikacji
# można zmienić kind na 'exponential'
df.ta.decay(append=True)

C:\Users\mt242\AppData\Local\Temp\ipykernel_21948\2913468828.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.ta.decay(append=True)
C:\Users\mt242\AppData\Local\Temp\ipykernel_21948\2913468828.py:3: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  df.ta.decay(append=True)


timestamp
2003-05-04 00:00:00+00:00    1.12169
2003-05-05 00:00:00+00:00    1.12924
2003-05-06 00:00:00+00:00    1.14234
2003-05-07 00:00:00+00:00    1.13494
2003-05-08 00:00:00+00:00    1.14820
                              ...   
2025-07-30 00:00:00+00:00    1.14278
2025-07-31 00:00:00+00:00    1.14217
2025-08-01 00:00:00+00:00    1.15830
2025-08-03 00:00:00+00:00    1.15876
2025-08-04 00:00:00+00:00    1.15838
Name: LDECAY_5, Length: 7154, dtype: float64

In [148]:
# 2. Heavily Weighted Close, ustawienia standardowe
# Default Inputs: na=?, nb=?, nc=?, nd=?, scalar=?, channel_eval=?, offset=0 <- parametry do modyfikacji
df.ta.hwc(append=True)

C:\Users\mt242\AppData\Local\Temp\ipykernel_21948\1952646224.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.ta.hwc(append=True)


,HWM,HWU,HWL
timestamp,,,
2003-05-04 00:00:00+00:00,1.121690,1.121690,1.121690
2003-05-05 00:00:00+00:00,1.123359,1.123359,1.123359
2003-05-06 00:00:00+00:00,1.127726,1.127726,1.127726
2003-05-07 00:00:00+00:00,1.129942,1.131802,1.128082
2003-05-08 00:00:00+00:00,1.134814,1.139761,1.129867
...,...,...,...
2025-07-30 00:00:00+00:00,1.152683,1.162194,1.143171
2025-07-31 00:00:00+00:00,1.147704,1.156786,1.138623
2025-08-01 00:00:00+00:00,1.147004,1.156171,1.137837


In [150]:
# 3. High-Low 2, ustawienia standardowe
# Default Inputs: offset=0 <- parametry do modyfikacji
df.ta.hl2(append=True)

timestamp
2003-05-04 00:00:00+00:00    1.122490
2003-05-05 00:00:00+00:00    1.125115
2003-05-06 00:00:00+00:00    1.136145
2003-05-07 00:00:00+00:00    1.137940
2003-05-08 00:00:00+00:00    1.140415
                               ...   
2025-07-30 00:00:00+00:00    1.148665
2025-07-31 00:00:00+00:00    1.143320
2025-08-01 00:00:00+00:00    1.149420
2025-08-03 00:00:00+00:00    1.158710
2025-08-04 00:00:00+00:00    1.156920
Name: HL2, Length: 7154, dtype: float64

In [152]:
# 4. High-Low-Close 3, ustawienia standardowe
# Default Inputs: talib=True, offset=0 <- parametry do modyfikacji
df.ta.hlc3(append=True)

timestamp
2003-05-04 00:00:00+00:00    1.122223
2003-05-05 00:00:00+00:00    1.126490
2003-05-06 00:00:00+00:00    1.138210
2003-05-07 00:00:00+00:00    1.136940
2003-05-08 00:00:00+00:00    1.143010
                               ...   
2025-07-30 00:00:00+00:00    1.146703
2025-07-31 00:00:00+00:00    1.142937
2025-08-01 00:00:00+00:00    1.152380
2025-08-03 00:00:00+00:00    1.158727
2025-08-04 00:00:00+00:00    1.157407
Name: HLC3, Length: 7154, dtype: float64

In [154]:
# 5. Kurtoza, ustawienia standardowe
# Default Inputs: length=30, offset=0 <- parametry do modyfikacji
df.ta.kurtosis(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    2.266997
2025-07-31 00:00:00+00:00    1.736120
2025-08-01 00:00:00+00:00    1.195612
2025-08-03 00:00:00+00:00    0.790561
2025-08-04 00:00:00+00:00    0.563689
Name: KURT_30, Length: 7154, dtype: float64

In [156]:
# 6. Logarithmic Return, ustawienia standardowe
# Default Inputs: length=20, cumulative=False, offset=0 <- parametry do modyfikacji
df.ta.log_return(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00    0.006708
2003-05-06 00:00:00+00:00    0.011534
2003-05-07 00:00:00+00:00   -0.006499
2003-05-08 00:00:00+00:00    0.011616
                               ...   
2025-07-30 00:00:00+00:00   -0.010688
2025-07-31 00:00:00+00:00   -0.000534
2025-08-01 00:00:00+00:00    0.014023
2025-08-03 00:00:00+00:00    0.000397
2025-08-04 00:00:00+00:00   -0.000328
Name: LOGRET_1, Length: 7154, dtype: float64

In [158]:
# 7. Mean Absolute Deviation, ustawienia standardowe
# Default Inputs: length=30, offset=0 <- parametry do modyfikacji
df.ta.mad(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    0.006407
2025-07-31 00:00:00+00:00    0.007475
2025-08-01 00:00:00+00:00    0.007801
2025-08-03 00:00:00+00:00    0.008062
2025-08-04 00:00:00+00:00    0.008112
Name: MAD_30, Length: 7154, dtype: float64

In [160]:
# 8. Mediana, ustawienia standardowe
# Default Inputs: length=30, offset=0 <- parametry do modyfikacji
df.ta.median(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.171975
2025-07-31 00:00:00+00:00    1.171975
2025-08-01 00:00:00+00:00    1.171255
2025-08-03 00:00:00+00:00    1.169815
2025-08-04 00:00:00+00:00    1.168985
Name: MEDIAN_30, Length: 7154, dtype: float64

In [162]:
# 9. Midpoint, ustawienia standardowe
# Default Inputs: length=?, talib=True, offset=0 <- parametry do modyfikacji
df.ta.midpoint(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00    1.125465
2003-05-06 00:00:00+00:00    1.135790
2003-05-07 00:00:00+00:00    1.138640
2003-05-08 00:00:00+00:00    1.141570
                               ...   
2025-07-30 00:00:00+00:00    1.148920
2025-07-31 00:00:00+00:00    1.142475
2025-08-01 00:00:00+00:00    1.150235
2025-08-03 00:00:00+00:00    1.158530
2025-08-04 00:00:00+00:00    1.158570
Name: MIDPOINT_2, Length: 7154, dtype: float64

In [164]:
# 10. Midprice, ustawienia standardowe
# Default Inputs: length=?, talib=True, offset=0 <- parametry do modyfikacji
df.ta.midprice(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00    1.125115
2003-05-06 00:00:00+00:00    1.132600
2003-05-07 00:00:00+00:00    1.136145
2003-05-08 00:00:00+00:00    1.140415
                               ...   
2025-07-30 00:00:00+00:00    1.149985
2025-07-31 00:00:00+00:00    1.148665
2025-08-01 00:00:00+00:00    1.149420
2025-08-03 00:00:00+00:00    1.149420
2025-08-04 00:00:00+00:00    1.157300
Name: MIDPRICE_2, Length: 7154, dtype: float64

In [166]:
# 11. Open-High-Low-Close 4, ustawienia standardowe
# Default Inputs: offset=0 <- parametry do modyfikacji
df.ta.ohlc4(append=True)

timestamp
2003-05-04 00:00:00+00:00    1.122350
2003-05-05 00:00:00+00:00    1.125270
2003-05-06 00:00:00+00:00    1.135960
2003-05-07 00:00:00+00:00    1.138250
2003-05-08 00:00:00+00:00    1.141025
                               ...   
2025-07-30 00:00:00+00:00    1.148793
2025-07-31 00:00:00+00:00    1.142897
2025-08-01 00:00:00+00:00    1.149828
2025-08-03 00:00:00+00:00    1.158820
2025-08-04 00:00:00+00:00    1.157745
Name: OHLC4, Length: 7154, dtype: float64

In [168]:
# 12. Price Distance, ustawienia standardowe
# Default Inputs: drift=1, offset=0 <- parametry do modyfikacji
df.ta.pdist(append=True)

timestamp
2003-05-04 00:00:00+00:00        NaN
2003-05-05 00:00:00+00:00    0.01235
2003-05-06 00:00:00+00:00    0.02256
2003-05-07 00:00:00+00:00    0.01408
2003-05-08 00:00:00+00:00    0.02842
                              ...   
2025-07-30 00:00:00+00:00    0.02206
2025-07-31 00:00:00+00:00    0.01035
2025-08-01 00:00:00+00:00    0.02495
2025-08-03 00:00:00+00:00    0.00414
2025-08-04 00:00:00+00:00    0.00742
Name: PDIST, Length: 7154, dtype: float64

In [170]:
# 13. Percent Return, ustawienia standardowe
# Default Inputs: length=20, cumulative=False, offset=0 <- parametry do modyfikacji
df.ta.percent_return(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00    0.006731
2003-05-06 00:00:00+00:00    0.011601
2003-05-07 00:00:00+00:00   -0.006478
2003-05-08 00:00:00+00:00    0.011683
                               ...   
2025-07-30 00:00:00+00:00   -0.010631
2025-07-31 00:00:00+00:00   -0.000534
2025-08-01 00:00:00+00:00    0.014122
2025-08-03 00:00:00+00:00    0.000397
2025-08-04 00:00:00+00:00   -0.000328
Name: PCTRET_1, Length: 7154, dtype: float64

In [172]:
# 14. Kwantyl, ustawienia standardowe
# Default Inputs: length=30, q=0, offset=0 <- parametry do modyfikacji
df.ta.quantile(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    1.171975
2025-07-31 00:00:00+00:00    1.171975
2025-08-01 00:00:00+00:00    1.171255
2025-08-03 00:00:00+00:00    1.169815
2025-08-04 00:00:00+00:00    1.168985
Name: QTL_30_0.5, Length: 7154, dtype: float64

In [174]:
# 15. Skewness, ustawienia standardowe
# Default Inputs: length=30, offset=0 <- parametry do modyfikacji
df.ta.skew(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00   -1.293123
2025-07-31 00:00:00+00:00   -1.344583
2025-08-01 00:00:00+00:00   -1.177894
2025-08-03 00:00:00+00:00   -1.024436
2025-08-04 00:00:00+00:00   -0.904881
Name: SKEW_30, Length: 7154, dtype: float64

In [176]:
# 16. Standard Deviation, ustawienia standardowe
# Default Inputs: length=30, ddof=1, talib=True, offset=0 <- parametry do modyfikacji
df.ta.stdev(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    0.008231
2025-07-31 00:00:00+00:00    0.009603
2025-08-01 00:00:00+00:00    0.009769
2025-08-03 00:00:00+00:00    0.009884
2025-08-04 00:00:00+00:00    0.009820
Name: STDEV_30, Length: 7154, dtype: float64

In [178]:
# 17. Variance, ustawienia standardowe
# Default Inputs: length=30, ddof=0, talib=True, offset=0 <- parametry do modyfikacji
df.ta.variance(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00    0.000068
2025-07-31 00:00:00+00:00    0.000092
2025-08-01 00:00:00+00:00    0.000095
2025-08-03 00:00:00+00:00    0.000098
2025-08-04 00:00:00+00:00    0.000096
Name: VAR_30, Length: 7154, dtype: float64

In [180]:
# 18. Z-Score, ustawienia standardowe
# Default Inputs: length=30, std=1, offset=0 <- parametry do modyfikacji
df.ta.zscore(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-30 00:00:00+00:00   -3.272635
2025-07-31 00:00:00+00:00   -2.774373
2025-08-01 00:00:00+00:00   -1.030557
2025-08-03 00:00:00+00:00   -0.925264
2025-08-04 00:00:00+00:00   -0.901273
Name: ZS_30, Length: 7154, dtype: float64

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7154 entries, 2003-05-04 00:00:00+00:00 to 2025-08-04 00:00:00+00:00
Columns: 283 entries, open to ZS_30
dtypes: float64(265), int32(16), int64(2)
memory usage: 15.3 MB
